In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Loading Data
df = pd.read_csv('E:\smart_internz\Data_collection\COVID-19_Sentiments.csv')

In [3]:
df.head()

,Text_Id,Text,Date,Location,Sentiments
0,1241032866567356417,RT @theskindoctor13: Shaheen Bagh is still on....,Fri Mar 20 16:04:27 +0000 2020,"Uttar Pradesh, India",0.000
1,1241032867699765249,RT @theskindoctor13: Shaheen Bagh is still on....,Fri Mar 20 16:04:27 +0000 2020,Mumbai,0.000
2,1241032875102703616,"RT @SmokingSkills_: Daughter of an IAS, son of...",Fri Mar 20 16:04:29 +0000 2020,"Jodhpur, India",0.350
3,1241032877099237379,RT @narendramodi: The young actors have someth...,Fri Mar 20 16:04:29 +0000 2020,"Gurugram, Bharat",0.125
4,1241032870405128192,RT @theskindoctor13: Shaheen Bagh is still on....,Fri Mar 20 16:04:28 +0000 2020,"New Delhi, India",0.000


In [4]:
# Modifying Date
df['Date'] = pd.to_datetime(df['Date']).dt.date

In [5]:
# Creating new_sentiments column out of polarities in sentiments
New_Sentiments_ls = []

for senti in df['Sentiments']:
    if senti < 0:
        New_Sentiments_ls.append(-1)
    elif senti > 0:
        New_Sentiments_ls.append(1)
    else:
        New_Sentiments_ls.append(0)

New_Sentiments_ls = pd.DataFrame(New_Sentiments_ls, columns=['New_Sentiments'])

df = pd.concat([df, New_Sentiments_ls], axis=1)

In [6]:
df.head()

,Text_Id,Text,Date,Location,Sentiments,New_Sentiments
0,1241032866567356417,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,"Uttar Pradesh, India",0.000,0
1,1241032867699765249,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,Mumbai,0.000,0
2,1241032875102703616,"RT @SmokingSkills_: Daughter of an IAS, son of...",2020-03-20,"Jodhpur, India",0.350,1
3,1241032877099237379,RT @narendramodi: The young actors have someth...,2020-03-20,"Gurugram, Bharat",0.125,1
4,1241032870405128192,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,"New Delhi, India",0.000,0


## Cleaning the text

In [7]:
# Cleaning the text
def func(x):
    try:
        trial = x.split(':')[1]
        return trial
    except:
        return x
df['Regenerated_Text'] = df['Text'].apply(lambda x: func(x))

In [8]:
# Converting to lower case
df['Regenerated_Text'] = df['Regenerated_Text'].apply(lambda x: x.lower())

# Removing urls
df['Regenerated_Text'] = df['Regenerated_Text'].apply(lambda x: x.split('https')[0])
df['Regenerated_Text'] = df['Regenerated_Text'].apply(lambda x: x.split('http')[0])

# Converting #word into just word
df['Regenerated_Text'] = df['Regenerated_Text'].str.replace('#', '')

# Remove @username
df['Regenerated_Text'] = df['Regenerated_Text'].str.replace('@[^\s]+','')

# Removing unwanted char
df['Regenerated_Text'] = df['Regenerated_Text'].str.replace("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ")    

# Remove additional white spaces
df['Regenerated_Text'] = df['Regenerated_Text'].str.replace('[\s]+', ' ')

In [ ]:
df.head()